# Testing
### Aufenthaltsdauer, nach Alter, Geschlecht, Herkunft, Stadtkreis, seit 1993
Datum: 15.03.2022




### Importiere die notwendigen Packages

In [42]:
#%pip install geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly leafmap

In [43]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

import datetime
import geopandas as gpd
import folium 
import plotly.express as px
import seaborn as sns
import leafmap

import requests
import io


In [44]:
SSL_VERIFY = False
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [45]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [46]:
#pd.options.display.float_format = lambda x : '{:,.1f}'.format(x) if (np.isnan(x) | np.isinf(x)) else '{:,.0f}'.format(x) if int(x) == x else '{:,.1f}'.format(x)
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import datetime und from datedtime import datetime, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [47]:
#today_date = datetime.date.today()
#date_time = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [48]:
#now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
    'datenstand: ', selectedMonat,
     int_times)


2022 3 datenstand:  1 time.struct_time(tm_year=2022, tm_mon=3, tm_mday=16, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=75, tm_isdst=-1)


Berechne die Variable Epoche um später das SAS-Datum in ein Unix-Datum umzuwandeln. Bei SAS beginnt die Epoche am 1.1.1960. Bei Unix am 1.1.1970.
Diese Variable wird beim CSV-Import benötigt.

In [49]:
epoch = datetime.datetime(1960, 1, 1)

### Setze einige Pfadvariabeln

- Der Packagename ist eigentlich der **Verzeichnisname** unter dem die Daten und Metadaten auf der Dropzone abgelegt werden.
- Definiert wird er bei SASA-Prozessen auf dem **Produkte-Sharepoint ([Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/PersonalViews.aspx?PageView=Personal&ShowWebPart={6087A3E7-8AC8-40BA-8278-DECFACE124FF}))**.
- Der Packagename wird auf CKAN teil der URL, daher ist die exakte Schreibweise wichtig.

Beachte: im Packagename müssen alle Buchstaben **klein** geschrieben werden. Dies weil CKAN aus grossen kleine Buchstaben macht.

**BITTE HIER ANPASSEN**

In [51]:
package_name = "bev_aufenthaltsdauer_bestand_alter_geschlecht_herkunft_stadtkreis_od5241"

In [52]:
dataset_name = "BEV524OD5241.csv"

**Statische Pfade in DWH-Dropzones**

In [53]:
dropzone_path_integ = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH"

In [54]:
dropzone_path_prod = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH"

**Statische Pfade CKAN-URLs**

In [55]:
ckan_integ_url ="https://data.integ.stadt-zuerich.ch/dataset/int_dwh_"

In [56]:
ckan_prod_url ="https://data.stadt-zuerich.ch/dataset/"

### Checke die Metadaten auf der CKAN INTEG- oder PROD-Webseite

Offenbar lassen sich aktuell im Markdownteil keine Variabeln ausführen, daher gehen wir wie unten gezeigt vor. Siehe dazu: https://data-dive.com/jupyterlab-markdown-cells-include-variables
Instead of setting the cell to Markdown, create Markdown from withnin a code cell! We can just use python variable replacement syntax to make the text dynamic

In [57]:
from IPython.display import Markdown as md

In [58]:
md(" **1. Dataset auf INTEG-Datakatalog:** Link {} ".format(ckan_integ_url+package_name))

 **1. Dataset auf INTEG-Datakatalog:** Link https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_aufenthaltsdauer_bestand_alter_geschlecht_herkunft_stadtkreis_od5241 

In [59]:
md(" **2. Dataset auf PROD-Datakatalog:** Link {} ".format(ckan_prod_url+package_name))

 **2. Dataset auf PROD-Datakatalog:** Link https://data.stadt-zuerich.ch/dataset/bev_aufenthaltsdauer_bestand_alter_geschlecht_herkunft_stadtkreis_od5241 

### Importiere einen Datensatz 

Definiere zuerst folgende Werte:
1) Kommt der Datensatz von PROD oder INTEG?
2) Beziehst Du den Datensatz direkt ab der DROPZONE oder aus dem INTERNET?

In [60]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "integ"; #prod vs something else
data_source = "web"; #dropzone vs something else
print(status+" - "+ data_source)

integ - web


In [61]:
# Filepath
if status == "prod":
    if data_source == "dropzone":
            fp = dropzone_path_prod+"\\"+ package_name +"\\"+dataset_name
            print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_prod_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)
else:
    if data_source == "dropzone":
        fp = dropzone_path_integ+"\\"+ package_name +"\\"+dataset_name
        print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_integ_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)


fp lautet:https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_aufenthaltsdauer_bestand_alter_geschlecht_herkunft_stadtkreis_od5241/download/BEV524OD5241.csv


Beachte, wie das SAS Datum (ohne Format) in ein UNIX Datum umgerechnet und als Datumsformat dargestellt wird! Siehe dazu `https://stackoverflow.com/questions/26923564/convert-sas-numeric-to-python-datetime`

In [62]:
# Read the data
if data_source == "dropzone":
    data2betested = pd.read_csv(
        fp
        , sep=','
        ,parse_dates=['StichtagDatJahr']
        ,low_memory=False
    )
    print("dropzone")
else:
    r = requests.get(fp, verify=False)  
    r.encoding = 'utf-8'
    data2betested = pd.read_csv(
        io.StringIO(r.text)
        ,parse_dates=['StichtagDatJahr']
        # KONVERTIERE DAS SAS DATUM IN EIN UNIXDATUM UND FORMATIERE ES
        #, date_parser=lambda s: epoch + datetime.timedelta(days=int(s))
        ,low_memory=False)
    print("web")

data2betested.dtypes

web


StichtagDatJahr    datetime64[ns]
AlterV20Kurz               object
AlterV20Sort                int64
SexCd                       int64
SexKurz                    object
HerkunftCd                  int64
HerkunftLang               object
KreisCd                     int64
KreisLang                  object
AufDauerP25               float64
AufDauerMedian            float64
AufDauerP75               float64
AufDauerMittel            float64
dtype: object

Berechne weitere Attribute falls notwendig

In [63]:
data2betested = (
    data2betested
    .copy()
    .assign(
        #Aktualisierungs_Datum_str= lambda x: x.Aktualisierungs_Datum.astype(str),
        StichtagDatJahr_str = lambda x: x.StichtagDatJahr.astype(str),
    )
    .sort_values('StichtagDatJahr', ascending=False)
    )
data2betested.dtypes

StichtagDatJahr        datetime64[ns]
AlterV20Kurz                   object
AlterV20Sort                    int64
SexCd                           int64
SexKurz                        object
HerkunftCd                      int64
HerkunftLang                   object
KreisCd                         int64
KreisLang                      object
AufDauerP25                   float64
AufDauerMedian                float64
AufDauerP75                   float64
AufDauerMittel                float64
StichtagDatJahr_str            object
dtype: object

### Einfache Datentests

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [64]:
#data2betested.head(6)

In [65]:
data2betested.dtypes

StichtagDatJahr        datetime64[ns]
AlterV20Kurz                   object
AlterV20Sort                    int64
SexCd                           int64
SexKurz                        object
HerkunftCd                      int64
HerkunftLang                   object
KreisCd                         int64
KreisLang                      object
AufDauerP25                   float64
AufDauerMedian                float64
AufDauerP75                   float64
AufDauerMittel                float64
StichtagDatJahr_str            object
dtype: object

In [66]:
data2betested.shape

(264, 14)

Beschreibe einzelne Attribute

In [67]:
data2betested.describe()

,AlterV20Sort,SexCd,HerkunftCd,KreisCd,AufDauerP25,AufDauerMedian,AufDauerP75,AufDauerMittel
count,264,264,264,264,264,264,264,264
mean,3,2,1,7,16,25,34,25
std,2,1,1,3,18,22,23,20
min,1,1,1,1,0,1,2,2
25%,2,1,1,4,3,8,13,9
50%,3,2,1,7,10,19,28,21
75%,5,2,2,10,26,41,52,40
max,6,2,2,12,104,104,104,104


Wie viele Nullwerte gibt es im Datensatz?

In [68]:
data2betested.isnull().sum()

StichtagDatJahr        0
AlterV20Kurz           0
AlterV20Sort           0
SexCd                  0
SexKurz                0
HerkunftCd             0
HerkunftLang           0
KreisCd                0
KreisLang              0
AufDauerP25            0
AufDauerMedian         0
AufDauerP75            0
AufDauerMittel         0
StichtagDatJahr_str    0
dtype: int64

Welches sind die Quartiere ohne Werte bei AnzBestWir?

In [69]:
data2betested[np.isnan(data2betested.AufDauerMittel)]

,StichtagDatJahr,AlterV20Kurz,AlterV20Sort,SexCd,SexKurz,HerkunftCd,HerkunftLang,KreisCd,KreisLang,AufDauerP25,AufDauerMedian,AufDauerP75,AufDauerMittel,StichtagDatJahr_str


### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [70]:
data2betested = data2betested.set_index("StichtagDatJahr")

In [71]:
data2betested.info()
data2betested.index.year.unique()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 264 entries, 2021-01-01 to 2021-01-01
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AlterV20Kurz         264 non-null    object 
 1   AlterV20Sort         264 non-null    int64  
 2   SexCd                264 non-null    int64  
 3   SexKurz              264 non-null    object 
 4   HerkunftCd           264 non-null    int64  
 5   HerkunftLang         264 non-null    object 
 6   KreisCd              264 non-null    int64  
 7   KreisLang            264 non-null    object 
 8   AufDauerP25          264 non-null    float64
 9   AufDauerMedian       264 non-null    float64
 10  AufDauerP75          264 non-null    float64
 11  AufDauerMittel       264 non-null    float64
 12  StichtagDatJahr_str  264 non-null    object 
dtypes: float64(4), int64(4), object(5)
memory usage: 28.9+ KB


Int64Index([2021], dtype='int64', name='StichtagDatJahr')

### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

In [72]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

In [74]:
data2betested.loc["2021"].head(2)
#data2betested.loc["2021-10-31":"2021-11-30"].head(2)

,AlterV20Kurz,AlterV20Sort,SexCd,SexKurz,HerkunftCd,HerkunftLang,KreisCd,KreisLang,AufDauerP25,AufDauerMedian,AufDauerP75,AufDauerMittel,StichtagDatJahr_str
StichtagDatJahr,,,,,,,,,,,,,
2021-01-01,0-19,1,1,M,1,Schweizer/in,1,Kreis 1,3,9,13,9,2021-01-01
2021-01-01,60-79,4,2,W,2,Ausländer/in,2,Kreis 2,8,22,38,24,2021-01-01


### Visualisierungen nach Zeitausschnitten

In [75]:
data2betested.columns

Index(['AlterV20Kurz', 'AlterV20Sort', 'SexCd', 'SexKurz', 'HerkunftCd', 'HerkunftLang', 'KreisCd',
       'KreisLang', 'AufDauerP25', 'AufDauerMedian', 'AufDauerP75', 'AufDauerMittel',
       'StichtagDatJahr_str'],
      dtype='object')

#### Mittlere Aufenthaltsdauer nach Altersgruppe, 2021

In [79]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAgg = data2betested\
    .groupby(['AlterV20Kurz', 'AlterV20Sort','KreisCd','KreisLang',]) \
    .agg(mean_WBev=('AufDauerMittel', 'mean')) \
    .sort_values('AlterV20Sort', ascending=True) 

myAgg.reset_index().head(3)

,AlterV20Kurz,AlterV20Sort,KreisCd,KreisLang,mean_WBev
0,0-19,1,1,Kreis 1,6
1,0-19,1,12,Kreis 12,8
2,0-19,1,10,Kreis 10,7


In [95]:
myTitle="Mittlere Aufenthaltsdauer nach Altersgruppe, 2021"

highlight = alt.selection(type='single', on='mouseover',
                          fields=['AlterV20Kurz'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(myAgg.reset_index().query('mean_WBev>0').sort_values('AlterV20Sort', ascending=True), title=myTitle).encode(
    x=alt.X('KreisCd', axis=alt.Axis(title='Stadtkreis'))# , axis=alt.Axis(format='%', title='percentage')
    , y=alt.X('mean_WBev', axis=alt.Axis(title='Mean Aufenthaltsdauer'))
    , color=alt.Color('AlterV20Kurz', legend=alt.Legend(title="Altersgruppen", orient="right"))  
    ,tooltip=['KreisLang', 'AlterV20Kurz','mean_WBev']    
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(4))
).interactive()

lines + points

alt.LayerChart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=464&Source=http%3A%2F%2Fkollaboration%2Eintranet%2Estzh%2Ech%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs%2FPersonalViews%2Easpx%3FPageView%3DPersonal%26ShowWebPart%3D%7B6087A3E7%2D8AC8%2D40BA%2D8278%2DDECFACE124FF%7D%23InplviewHash6087a3e7%2D8ac8%2D40ba%2D8278%2Ddecface124ff%3DSortField%253DPackagename%2DSortDir%253DAsc%2DFilterField1%253DDiffusionsereignis%2DFilterValue1%253DBev%2525C3%2525B6lkerung%2525202020)**